## Домашнее творческое задание по дисциплине "Технологии обработки больших данных" (часть 2)

Выполнил: студент группы __ПМ20-2 Короткова Вера Владимировна__

Вариант: __53__

    

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

__В задачах, связанных с пакетом `dask`, явно указано, какой коллекцией вы должны пользоваться при решении задачи. Все операции вычислений должны проводиться  над этой коллекцией и средствами пакета `dask`, если в задании не сказано обратного.__ 



__Задание 1__. В файле `scores.npy` содержится массив оценок рейтингов вопросов. Напишите функцию, которая считает сумму квадратов цифр числа. Векторизуйте эту функцию при помощи `numba.vectorize` и посчитайте количество чисел из файла, сумма квадратов цифр которых меньше 50.

In [1]:
import numpy as np

scores = np.load('scores.npy')

In [2]:
def square_sum(n):
    s = 0
    while n//10 > 0:
        s += (n%10)**2
        n = n//10
    s += (n%10)**2
    return s

In [3]:
square_sum(10)

1

In [4]:
import numba
square_sum_v = numba.vectorize()(square_sum)

In [5]:
(square_sum_v(scores) < 50).sum()

565751

__Задание 2__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Создайте базу данных SQLite с таблицей `Questions`. Напишите и выполните запрос, который возвращает ID пользователей, у которых есть хоть один вопрос со score, равным 1. Для выполнения запроса используйте функцию `pd.read_sql_query` 

In [6]:
import pandas as pd
questions = pd.read_csv('questions.csv')

In [7]:
import sqlite3

In [8]:
con = sqlite3.connect("dtz.sqlite")
cur = con.cursor()

In [9]:
creation = '''
CREATE TABLE Questions(
    Id int,
    OwnerUserId int,
    CreationDate varchar,
    Score int,
    Title varchar,
    Body varchar
);
'''

In [10]:
cur.executescript(creation)
con.commit()

In [11]:
insertion = '''
 INSERT INTO Questions(Id, OwnerUserId, CreationDate, Score, Title, Body) VALUES (?, ?, ?, ?, ?, ?)
'''

In [12]:
cur.executemany(insertion, questions.values)
con.commit()

In [13]:
pd.read_sql_query("SELECT * FROM Questions", con)

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...
...,...,...,...,...,...,...
607277,40143190,333403.0,2016-10-19T23:36:01Z,1,How to execute multiline python code from a ba...,<p>I need to extend a shell script (bash). As ...
607278,40143228,6662462.0,2016-10-19T23:40:00Z,0,How to get google reCaptcha image source using...,<p>I understood that reCaptcha loads a new fra...
607279,40143267,4064680.0,2016-10-19T23:44:07Z,0,Updating an ManyToMany field with Django rest,<p>I'm trying to set up this API so I can use ...
607280,40143338,7044980.0,2016-10-19T23:52:27Z,2,Most possible pairs,"<p>Given a list of values, and information on ..."


In [14]:
pd.read_sql_query(
    '''
    SELECT OwnerUserId 
    FROM Questions
    GROUP BY OwnerUserId
    HAVING min(Score)=1
    ''', con)

,OwnerUserId
0,25
1,50
2,56
3,96
4,145
...,...
40380,7044156
40381,7044483
40382,7044689
40383,7044759


__Задание 3__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Используя `multiprocessing.Pool` выясните, сколько вопросов задал каждый из пользователей. При решении задачи разделите файл на части, для обработки каждой из частей используйте параллельные вычисления.

In [15]:
import multiprocessing

In [16]:
for index, chunk in enumerate(pd.read_csv('questions.csv', chunksize=questions.shape[0]//6 + 1)):
    chunk.to_csv(f'questions_{index+1}.csv', index=False)

In [17]:
files = [f"questions_{index}.csv" for index in np.arange(1, 7)]

In [18]:
%%file count_questions.py

import pandas as pd

def count_questions(file):
    file = pd.read_csv(file)
    return file.groupby('OwnerUserId').count()['Body']

Writing count_questions.py


In [19]:
from count_questions import count_questions

In [24]:
with multiprocessing.Pool(processes=6) as pool:
    res = pool.map(count_questions, files)

In [25]:
res = pd.concat(res)
res.groupby('OwnerUserId').count()

OwnerUserId
25.0         1
26.0         1
30.0         1
38.0         1
48.0         1
            ..
7044946.0    1
7044959.0    1
7044979.0    1
7044980.0    1
7044992.0    1
Name: Body, Length: 213927, dtype: int64

__Задание 4__. В файле `qembeddings.hdf` находятся векторные представления заголовков вопросов, заданные на сайте stackoverflow. В файле `qtitles.pickle` находится список заголовков этих вопросов. Напишите функцию, которая для вопроса с индексом `i` выводит заголовки 5 ближайших вопросов в смысле расстояния между эмбеддингами этих вопросов. Выберите метрику, наиболее подходящую (на ваш взгляд) для решения этой задачи. Для работы с эмбеддингами используйте `dask.array`.

In [26]:
import dask.array as da
import h5py
import dask

hdf = h5py.File('qembeddinbgs.h5', "r")
dset = hdf['embeddings']

In [27]:
qembeddinbgs = da.from_array(dset)

In [28]:
import pickle

with open('qtitles.pickle', 'rb') as f:
    qtitles = pickle.load(f)    

In [20]:
def top_five(i, qembeddinbgs):
    qtrans = qembeddinbgs.transpose()

    qnorm = qembeddinbgs[i]/da.linalg.norm(qembeddinbgs[i])
    qtransnorm = qtrans/(da.linalg.norm(qtrans, axis=1).reshape(-1, 1))

    cosdistance = 1 - da.dot(qnorm, qtransnorm)
    
    return da.array(qtitles)[cosdistance.argtopk(-6)[1:]].compute()

In [21]:
top_five(1, qembeddinbgs)

array(['Set timezone to pandas dataframe',
       'Date parsing and timezone adjusting in pandas dataframes',
       'Converting time zone pandas dataframe',
       'Split datetime64 column into a date and time column in pandas dataframe',
       'Combine date column and time column into index in pandas data frame'],
      dtype='<U172')

__Задание 5__. В файлах `questions_with_answers_*.jsonl` содержатся вопросы, заданные на сайте stackoverflow, и ответы на них. Посчитайте и выведите на экран количество вопросов, на которые дали более 10 ответов. Для обработки данных используйте `dask.bag`.

In [30]:
import dask.bag as db
import json

def to_dict(data):
    try:
        return json.loads(data)
    except:
        return {'answers': []}

In [31]:
q_ans = db.read_text('questions_with_answers/questions_with_answers_*.jsonl').map(to_dict)

In [36]:
len(q_ans.filter(lambda x: len(x['answers']) > 10).compute())

659

__Задание 6__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Для вопросов, рейтинг которых больше медианного значения по столбцу, найдите топ-5 самых часто используемых слов во всех вопросах в сокупности. Для обработки данных используйте `dask.dataframe`.

In [49]:
import dask.dataframe as dd
questions = dd.read_csv('questions.csv')

In [56]:
med = questions['Score'].median_approximate()
med.compute()

2.0

In [120]:
questions_median = questions[questions['Score'] > med]

In [142]:
words = questions_median['Body'].map(lambda x: re.sub(r'[(<.*?>)=]', '', x)).str.split().explode()

In [144]:
words.value_counts().compute()[:5]

the    565188
to     424904
a      347930
I      314827
in     271519
Name: Body, dtype: int64